In [1]:
using Plots, Interact, Dates, DataFrames, LinearAlgebra, IterativeSolvers
using Netatmo

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-2376801219160278957\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-7768021887231988140\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio/main\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n        console.warn(\"Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).\");\n        require([\"/assetserver/15ffaef09c6581996e3c9c7ce400ef5d8e046a28-dist/main.js\"], function (webIOModule) {\n            webIOModule.load_ipython_extension();\n        });\n        warning_div.innerHTML = \"<strong>Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).</strong>\";\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<div\n    id=\"webio-warning-2376801219160278957\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    <!-- TODO: link to installation docs. -->\n</div>\n")

CSV_ARCHIVE=/lustre/storeB/users/roels/netatmo
JSON_ARCHIVE=/lustre/storeB/project/metproduction/products/netatmo/


In [2]:
dtg       = DateTime(2019,5,1,0)
period    = Hour(24)
timerange = dtg:Minute(10):dtg + period
latrange  = 59.9:0.01:60  
lonrange  = 10.7:0.01:10.8
df = Netatmo.read(timerange, latrange=latrange, lonrange=lonrange)

Reading: CSV files 100%|████████████████████████████████| Time: 0:00:26


,id,time_utc
,String,Int64
1,enc:16:8USjqWB8mr5di7cAMak+ZVA4I5ogvXj4Hyjl0re2lkAa211EN/omZxnkBFiCLGgW,1556668804
2,enc:16:PhrU/F5+73GUzqtRppynU1s28nKNZIAGocaKZjHkP99W51WPX6uhdi7m0alPFnNb,1556668818
3,enc:16:Z9PjEIA+4bnjrGy9ZQgtdBgRF/pbezvz31wT0/vatkExyp0VwerAPreoawchlHtr,1556669073
4,enc:16:enUHiQxRLiIB6A+qJrjGnVWhMKwSzPMv6sJ0VN2lSLqtqYkbjoUQeWKE4InWhTvO,1556669093
5,enc:16:7RKDJoK+UWw9QPF187w5sE8lRlABnUCO1oPLjOTsd26wY5LgsPZYnexaiHGHH5pJ,1556669084
6,enc:16:pRZIyoowNLqEr6lZFDBz1m50JiohnsowDxB38fVsVum4eIpmNWCA8Tc21tDCR7Ze,1556669258
7,enc:16:zEydIQ1EjqZ8915fG2Mum1Mnsecn7/Yp58ead3h1t249xC4+xQjcyVsSs/GknfRy,1556668936
8,enc:16:JL/v0BtE2i8ir5bZRnF/6tvAwwFozkEC+/58Q/Y7gWSzfncSh2PZ3u8TNV1pNqNF,1556669099
9,enc:16:CzKHwceirDxtkSzcgDbPdP0M7hB+n7WBwpCmFFkNviancSdzZ2ONCUvoE2tUaXlM,1556669237


In [42]:
    df.pressurehat = 0 
    groupbyid = groupby(df,:id) 
    s = groupbyid[1]
    σf = 10
    σₒ = 0.1
    r = 4
    # Kou(t₁,t₂)  = σf^2*exp(-1/2*abs(t₁-t₂)/(60*60*r))   # Ornstein–Uhlenbeck
    Kse(t₁,t₂) = σf^2*exp(-1/2*(t₁-t₂)^2/(60*60*r)^2)          # squared-exponential 
    K  = [Kse(t₁,t₂) for t₁ in s[:time_utc], t₂ in s[:time_utc] ] 
    # Ks = [rbf(t1,t2) for t1 in datetime2unix.(timerange),     t2 in s1[:time_utc] ]     
    KpI = copy(K)  
    KpI[diagind(KpI)] .= diag(KpI) .+ (σₒ)^2
    timecg  = @elapsed q, cglog = cg(KpI,s[:pressure],log=true)           
    pshat = K*q
    groupbyid[1][:pressurehat] = pshat

143-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [43]:
k(x1,x2) = Kse(x1.x,x2.x)* Kse(x1.y,x2.y)*Kse(x1.t,x2.t)

k (generic function with 1 method)